In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from pathlib import Path
from pprint import pprint

import requests
from IPython.core.display import HTML
from pyspark.sql import SparkSession

from spark_surveymonkey import transform_surveymonkey

display(HTML("<style>pre { white-space: pre !important; }</style>"))
spark = SparkSession.builder.appName('spark-surveymonkey').getOrCreate()

In [3]:
with open('credentials.json') as f:
    credentials = json.load(f)

TOKEN = credentials['token']


def download_survey(survey_id, endpoint, directory):
    host = 'https://api.surveymonkey.net/v3'
    url = f'{host}/surveys/{survey_id}/{endpoint}'
    headers = {
        'Authorization': f'Bearer {TOKEN}',
        'Content-Type': 'application/json',
    }
    response = requests.get(url, headers=headers).json()
    Path(directory).mkdir(parents=True, exist_ok=True)
    # TODO download multiple pages
    with open(f'{directory}/{endpoint.replace("/", "_")}.json', 'w') as f:
        json.dump(response, f)


In [4]:
survey_id = 283257051
dir_ = Path('data', str(survey_id))

download_survey(survey_id, 'responses/bulk/?per_page=100', Path(dir_, 'responses'))
download_survey(survey_id, 'details', dir_)

with open(Path(dir_, 'details.json')) as f:
    details = json.load(f)

df = transform_surveymonkey(spark, Path(dir_, 'responses'), details)

In [ ]:
pprint(df.columns)

In [ ]:
df.show()